In [1]:
# libraries
using StatsBase
using Combinatorics
using AbstractTrees
using GraphRecipes
using Plots

default(size=(1000, 1000))

In [2]:
mutable struct Node
    itemset # ex: ['a', 'b']
    children::Vector{Any}
    sup 
    parent 

    # constructor
    Node(its, chi = [], su = 0, par = nothing) = new(its, chi, su, par)
end

# HÀM BỔ TRỢ

In [3]:
AbstractTrees.children(node::Node) = node.children

function AbstractTrees.printnode(io::IO, node::Node)
    str = node.itemset
    print(io, str)
end

function tree_to_file(tree, file_path)
    file_ins = open(file_path, "w") # file instance
    maxdepth = 6000 # display maximum number of branches
    print_tree(file_ins, tree, maxdepth) # export tree to file
    close(file_ins)
end 

function read_database(file_path)
    database = []

    open(file_path) do f
        for (i, line) in enumerate(eachline(f))
            row = [parse(Int64, x) for x in split(line)]
            push!(database, row)
        end
    end
    return database
end

function print_candidate_nodes(candidate_nodes)
    for node in candidate_nodes
        println("  $(node.itemset) -> $(node.children)")
    end
end

function large_itemsets_to_file(all_large_itemset, file_path)
    open(file_path, "w") do file_ins
        for itemset in all_large_itemset
            for set in itemset
                println(file_ins, set)
            end
        end
    end
end

large_itemsets_to_file (generic function with 1 method)

# TREE PROJECTION

In [4]:
function candidate_nodes_gen(large_itemset_nodes)
    large_itemset = [node.itemset for node in large_itemset_nodes]
    candidate_nodes = []
    k = length(large_itemset[1]) + 1 # number of items in each candidate

    # for every different pair 
    for i=1:length(large_itemset)
        for j=i+1:length(large_itemset)
            # if the pair doesn't have the same first k-2 items -> cannot merge to create new candidate 
            if large_itemset[i][1:k-2] != large_itemset[j][1:k-2]
                continue
            end

            # else, merge the 2 nodes into u 
            u = union(Set(large_itemset[i]), Set(large_itemset[j]))
            # then convert u (Set) to u (Array), sort u to guarantee a lexico order
            u_converted = sort!((x->(x)).(u)) 

            # pruning: 
            flag = 0 
                
            for subset in collect(combinations(u_converted,k-1))
                # subset generated by combinations() might not be lexicographically ordered so re-sort the subset
                sort!(subset) 
                
                # if ∃ subset, subset is not a large itemset 
                if subset ∉ large_itemset 
                    flag = 1
                    break
                end
            end
            # otherwise, if ∀ subset, subset is a large itemset
            if flag == 0
                node = Node(u_converted)
                node.parent = large_itemset_nodes[i] # form child-parent relationship
                push!(large_itemset_nodes[i].children, node) # form parent-child relationship
                push!(candidate_nodes, node) # add node to return list 
            end
        end  
    end
    return candidate_nodes
end

candidate_nodes_gen (generic function with 1 method)

In [5]:
function apriori_tree(database, min_sup)
    min_sup = min_sup * length(database)
    one_itemset = sort(collect(countmap(collect(Iterators.flatten(database)))), by=x->x[1]) # lexical sort 
    one_itemset = [Node([item.first], [], item.second) for item in one_itemset if item.second >= min_sup]
    tree = Node([nothing], one_itemset) # init: tree has only 1 root node and some 1-itemset nodes
    itr = tree.children # root_node iterator

    while !isempty(itr)
        println("generating candidates...")
        candidate_nodes = candidate_nodes_gen(itr) 
        
        # print message
        try
            println(length(candidate_nodes[1].itemset))
        catch
            # pass
        end
        
        # update support of all candidates
        for transaction in database
            for candidate in candidate_nodes 
                if candidate.itemset ⊆ transaction # if transaction contains candidate 
                    candidate.sup += 1
                end
            end
        end
        
        # filter large itemset nodes, 
        large_itemset = []
        for node in candidate_nodes
            if node.sup >= min_sup
                push!(large_itemset, node)
            else # if there exists an unlarge itemset in tree, delete it from tree 
                par = node.parent 
                deleteat!(par.children, findall(x->x==node, par.children))
                node.parent = []
            end
        end
        
        itr = large_itemset # update itr pointer
    end

    println("Done")
    return tree 
end

apriori_tree (generic function with 1 method)

# APRIORI

In [6]:
function candidate_gen(large_itemset)
    candidate_list = []  # return list
    k = length(large_itemset[1]) + 1 # number of items in each candidate

    # for every pair in large_itemset
    for i=1:length(large_itemset)
        for j=i+1:length(large_itemset)
            # if the pair doesn't have the same first k-2 items -> cannot merge to create new candidate 
            if large_itemset[i][1:k-2] != large_itemset[j][1:k-2]
                continue
            end
            
            # merge the pair into u 
            u = union(Set(large_itemset[i]), Set(large_itemset[j]))
            # convert u (Set) to u (Array), then sort it to guarantee a lexico order to prevent duplicate 
            u_converted = sort!((x->(x)).(u)) 
            
            # pruning
            flag = 0 
            
            for subset in collect(combinations(u_converted,k-1))
                # subset generated by combinations() might not be lexicographically ordered so re-sort the subset
                # ex: ['C','T'] ∈ [['A','B'], ['C', 'T']] while  ['T','C'] ∉ [['A','B'], ['C', 'T']]
                sort!(subset) 
                
                # if ∃ subset, subset is not a large itemset 
                if subset ∉ large_itemset 
                    flag = 1
                    break
                end
            end
            
            # otherwise, if all subsets are all large itemsets
            if flag == 0
                push!(candidate_list, u_converted) # append u_converted to return list 
            end
        end
    end
    return candidate_list
end

candidate_gen (generic function with 1 method)

In [7]:
function apriori(database, min_sup)
    min_sup = min_sup * length(database)
    # init: take all the 1-itemsets that satisfies min_sup
    one_itemset = countmap(collect(Iterators.flatten(database)))
    one_itemset = collect(keys(filter(kv -> kv.second >= min_sup, one_itemset)))  
    one_itemset = sort([[item] for item in one_itemset])
    large_itemset = one_itemset 
    # result list 
    all_large_itemset = [] 
    
    while(!isempty(large_itemset))
        push!(all_large_itemset, large_itemset)
        println("Generating canididates...")
        candidate_list = candidate_gen(large_itemset) # generate all candidates 
        freq = Dict(cand=>0 for cand in candidate_list) # init freq to count sup 

        # print message
        try 
            println(length(collect(candidate_list)[1]))
        catch
            # pass
        end
        
        # update support of all candidates
        for transaction in database
            for candidate in candidate_list
                if candidate ⊆ transaction
                    freq[candidate] += 1
                end
            end
        end

        # filter large itemsets from candidates
        large_itemset = collect(keys(filter(kv -> kv.second >= min_sup, freq)))
    end 

    println("Done")
    return all_large_itemset
end

apriori (generic function with 1 method)

# CHẠY THUẬT TOÁN VỚI CÁC DATASET

In [8]:
# sample testcase 
sample_database = [["A" "C" "T" "W"], 
                    ["C" "D" "W"],
                    ["A" "C" "T" "W"],
                    ["A" "C" "D" "W"],
                    ["A" "C" "D" "T" "W"],
                    ["C" "D" "T"]]
min_sup = 0.5 
res = apriori(sample_database, min_sup)
large_itemsets_to_file(res, "sample_apriori.txt") 
# # ------------------------------------------------
res = apriori_tree(sample_database, min_sup) 
tree_to_file(res, "sample_tree.txt") 

Generating canididates...


2

Generating canididates...
3
Generating canididates...
4
Generating canididates...


Done


generating candidates...


2


generating candidates...
3
generating candidates...
4
generating candidates...
Done


In [ ]:
database = read_database("foodmart.txt")
min_sup = 0.000965 # 10 (~15m) -> 0.00241; 4 (~21m) -> 0.000965

# output: 1165 large itemsets 
res = apriori(database, min_sup) 
large_itemsets_to_file(res, "foodmart_apriori.txt") 
# ------------------------------------------------
res = apriori_tree(database, min_sup) 
tree_to_file(res, "foodmart_tree.txt") 

In [195]:
database = read_database("chess.txt")
min_sup = 0.9

# output: 622 large itemsets 
res = apriori(database, min_sup)
large_itemsets_to_file(res, "chess_apriori.txt") # export tree to foodmart_op.txt
# ------------
res = apriori_tree(database, min_sup) 
tree_to_file(res, "chess_tree.txt") 

Generating canididates...


2


Generating canididates...
3


Generating canididates...
4


Generating canididates...
5


Generating canididates...
6


Generating canididates...
7
Generating canididates...
Done


In [188]:
database = read_database("mushrooms.txt")
min_sup = 0.4

# output: 505 large itemsets
res = apriori(database, min_sup) 
large_itemsets_to_file(res, "mushrooms_apriori.txt") # export tree to foodmart_op.txt
# ------------
res = apriori_tree(database, min_sup) 
tree_to_file(res, "mushrooms_tree.txt") 

Generating canididates...
2


Generating canididates...
3


Generating canididates...
4


Generating canididates...
5


Generating canididates...
6
Generating canididates...
Done
generating candidates...
2


generating candidates...
3


generating candidates...
4


generating candidates...
5


generating candidates...
6
generating candidates...
Done


In [189]:
database = read_database("retail.txt")
min_sup = 0.05

# output: ?
res = apriori(database, min_sup) 
large_itemsets_to_file(res, "retail_apriori.txt") # export tree to foodmart_op.txt
# ------------
res = apriori_tree(database, min_sup) 
tree_to_file(res, "retail_tree.txt") 

Generating canididates...
2


Generating canididates...
3
Generating canididates...
Done


generating candidates...
2


generating candidates...
3
generating candidates...
Done


# ÁP DỤNG THỰC TẾ
- Source: https://www.kaggle.com/irfanasrullah/groceries/version/2
- Mô tả file `groceries.csv`: chứa 9835 dòng/transaction, mỗi dòng tương ứng với giỏ hàng của 1 user mua hàng tại cửa hàng
- Bài toán: liệt kê các tập phổ biến. Rút luật?

In [ ]:
database = []

open("groceries.csv") do f
    for (i, line) in enumerate(eachline(f))
        row = [string(x) for x in split(line, ",")]
        push!(database, row)
    end
end
database

In [43]:
res = apriori_tree(database, 0.01)
tree_to_file(res, "groceries_tree.txt") 

generating candidates...
2


generating candidates...
3


generating candidates...
4
Done
